In [1]:
import pickle 
import pandas as pd
import spacy
from tqdm.auto import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.notebook import tqdm
tqdm.pandas()
import re
import transformers
from sentence_transformers import SentenceTransformer, util
import nltk
from nltk import tokenize
import pickle
import urllib.request as requests
import json
import statistics
import torch

In [2]:
nltk.download('punkt')
nlp = spacy.load("en_core_web_lg")

[nltk_data] Downloading package punkt to /Users/haskar140/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
model = SentenceTransformer('all-mpnet-base-v2')

In [4]:
df=pd.read_csv('gmo_to_uniqueId.csv')
materialID=df['lastRunJob'].to_list()

In [5]:
def process_text(text):
    #doc = self.nlp(unicode(text.lower()))
    text=text.replace('\\n', ' ')
    doc = nlp(text.lower())
    result = []
    for token in doc:
        if token.text in nlp.Defaults.stop_words:
            continue
        if token.is_punct:
            continue
        if token.lemma_ == '-PRON-':
            continue
        result.append(token.lemma_)
    return nlp(" ".join(result))

In [6]:
def filterText( doc_ref, posOfInterest = ["NOUN", "VERB"]):

    trash_list = []
    for entity in doc_ref.ents:
        for en in entity.text.split(" "):
            trash_list.append(en)

    #print(trash_list)

    ## add nouns and verbs
    new_doc_ref = []
    for token in doc_ref:
        #print(token.text, token.pos_)
        #if token.pos_ in ["VERB", "NOUN"]:
        if token.pos_ in posOfInterest:
            if token.text not in trash_list:
                #print(token.text)
                if len(token.text)>1:
                    new_doc_ref.append(token.text.replace("\'", "").replace("\"", ""))

    ## add useful name entities
    for entity in doc_ref.ents:
        if entity.label_ not in ["DATE", "TIME", "PERCENT", "MONEY", "QUANTITY", "ORDINAL", "CARDINAL"]:
            if entity.text not in new_doc_ref:
                #print(entity.text, entity.label_)
                new_doc_ref.append(entity.text.replace("\'", "").replace("\"", ""))

    ## remove single-letter words
    for text in new_doc_ref:
        if (len(text)<=1):
            new_doc_ref.remove(text)

    return nlp(" ".join(new_doc_ref))

In [ ]:
with open("segmentedinput.pkl", "rb") as pkl_handle:
	output = pickle.load(pkl_handle)

In [ ]:
print(len(output['FILE_MAF_20220213T082327Z_GMO_00000000030342_01']))

In [ ]:
cc=output['FILE_MAF_20220213T082327Z_GMO_00000000030342_01'][0]['ClosedCaption']
cc=process_text(cc)
cc=filterText(cc)
print(cc)

In [ ]:
cc=list(cc)

In [ ]:
for i in range(len(cc)):
    cc[i]=cc[i].text.replace("♪", "")
    cc[i]=cc[i].replace("   ", "")


In [ ]:
print(len(cc))


In [ ]:
onlylabels=df1['Name'].to_list()

In [ ]:

emb1=model.encode(cc) #, convert_to_tensor=True)
emb2=model.encode(onlylabels) #, convert_to_tensor=True)
cos_sim=util.cos_sim(emb1,emb2)

all_sentence_combinations = []
for i in range(cos_sim.shape[0]):
    for j in range(cos_sim.shape[1]):
        all_sentence_combinations.append([cos_sim[i][j], i, j])

#Sort list by the highest cosine similarity score
all_sentence_combinations = sorted(all_sentence_combinations, key=lambda x: x[0], reverse=True)


In [ ]:
print(cc[116])
print(onlylabels[28])

In [ ]:
print(all_sentence_combinations)

In [ ]:
metalist=[]
for id in tqdm(materialID):
    segments=[]
    for i in range(len(output[id])):
        cc=output[id][i]['ClosedCaption']
        cc=process_text(cc)
        cc=filterText(cc)
        segments.append(cc)
    metalist.append(segments)

In [ ]:
with open('PreProcessed_segmented_spacy.pkl', 'wb') as f:
    pickle.dump(metalist, f)

In [7]:
with open('PreProcessed_segmented_spacy.pkl', 'rb') as f:
    newmetalist=pickle.load(f)

In [8]:
newmetalist[0]

[tell thing woman love go come go heat sensor go know guy company know sensor need maintenance work order let authorization hear gather evidence mean settlement kind tomorrow send district attorney story believe get believe happen happen mean go know think care people work paycheck paycheck life thing turn life guy understand day finalize happen word know night dinner party time know rail file room talk talk fact smoking gun document ask go mean think go ask time file room walk door buzz hell mean mean woman believe believe tell kill case put appreciate treat tell idiot idea know go tell go care know let engine turn talk talk case believe fight love love person give compliment suppose say love love say love feel know love knock door know way company let use work order find matter find answer question know difference win win know strongarm listen oppose counsel secretary poke file room look let accord go claim fraud theft number thing go make deal drop case go go file paperwork go let w

In [9]:
df1=pd.read_csv('InitialTargetDatabase.csv')    
df1.head()

,Unnamed: 0,Name,info
0,0,Automotive,sedan; convertible; wagon; suv; van; pick up...
1,1,Books and Literature,library bookstore Entertainment
2,2,Business and Finance,"Office environment, Cubicles, Desk + Screens, ..."
3,3,Education,university indoor; university outdoor; cla...
4,4,Events and Attractions,


In [10]:
def clean(text):
    text=text.replace("(", "")
    text=text.replace(")", "")
    text=text.replace(";", "")
    text=text.replace(",", "")
    text=text.replace("+", "")
    text=text.replace(".", "")
    return text

In [11]:
df1['Combined']=df1['Name']+' '+df1['info']

In [12]:
df1['Combined'].apply(clean)

0     Automotive   sedan convertible wagon suv van p...
1     Books and Literature     library bookstore Ent...
2     Business and Finance Office environment Cubicl...
3     Education     university indoor university out...
4                      Events and Attractions          
5     Family and Relationships family dinner family ...
6     Fine Art art history paintings sculptures ball...
7     Food & Drink   food and drink objects kitchen ...
8     Healthy Living gym working out smoothies hikin...
9     Hobbies & Interests       Entertainment Consum...
10    Home & Garden   bed couch chair dining table m...
11    Medical Health doctors hospitals doctors offic...
12    Movies Movie Theater Projection Screen popcorn...
13    Music and Audio Visual: instruments DJ table r...
14                Pets collar leash  dog cat   Retail  
15    Pop Culture social media entertainment news   ...
16    Real Estate For sale sign open house sign movi...
17    Religion & Spirituality religious books re

In [13]:
df1['Combined'].apply(process_text)


0     (automotive,    , sedan, convertible, wagon, s...
1     (book, literature,      , library, bookstore, ...
2     (business, finance, office, environment, cubic...
3     (education,      , university, indoor, univers...
4                        (event, attraction,          )
5     (family, relationship, family, dinner, family,...
6     (fine, art, art, history, painting, sculpture,...
7     (food, drink,    , food, drink, object, kitche...
8     (healthy, living, gym, work, smoothie, hiking,...
9     (hobby, interest,        , entertainment, cons...
10    (home, garden,    , bed, couch, chair, dining,...
11    (medical, health, doctor, hospital, doctor, of...
12    (movie, movie, theater, projection, screen, po...
13    (music, audio, visual, instrument, dj, table, ...
14    (pet, collar, leash,   , dog, cat,    , retail...
15    (pop, culture, social, medium, entertainment, ...
16    (real, estate, sale, sign, open, house, sign, ...
17    (religion, spirituality, religious, book, 

In [14]:
targetlist=df1['Combined'].to_list()

In [15]:
temp=targetlist[27]
temp=temp.replace('*News business use case needs to be scoped by NBCU - priority TBD', '')
targetlist[27]=temp

In [16]:
print(targetlist[7])

Food & Drink   food and drink objects kitchen indoor; restaurant indoor Restaurants, Consumer Goods  


In [17]:
listofepisodes=['FILE_MAF_20220215T074142Z_GMO_00000000368391_01', 'FILE_MAF_20220510T214825Z_GMO_00000000393602_01', 'FILE_MAF_20210805T224425Z_GMO_00000000196552_01', 'FILE_MAF_20220215T035724Z_GMO_00000000332330_01', 'FILE_MAF_20220321T225328Z_GMO_00000000354812_01', 'FILE_MAF_20210805T224839Z_GMO_00000000296761_01', 'FILE_MAF_20220510T231142Z_GMO_00000000398884_01', 'FILE_MAF_20211222T181429Z_GMO_00000000007508_01', 'FILE_MAF_20220213T082327Z_GMO_00000000030342_01', 'FILE_MAF_20210809T200034Z_GMO_00000000171494_01', 'FILE_MAF_20211221T215926Z_GMO_00000000322097_01', 'FILE_MAF_20220215T092524Z_GMO_00000000390298_01', 'FILE_MAF_20210728T172450Z_GMO_00000000001655_01', 'FILE_MAF_20220212T185028Z_GMO_00000000007263_01', 'FILE_MAF_20210807T072641Z_GMO_00000000015474_01', 'FILE_MAF_20210730T000327Z_GMO_00000000346793_01', 'FILE_MAF_20211106T183216Z_GMO_00000000345589_01', 'FILE_MAF_20210729T141453Z_GMO_00000000321307_01', 'FILE_MAF_20210729T092524Z_GMO_00000000044319_01', 'FILE_MAF_20220321T225323Z_GMO_00000000347371_01', 'FILE_MAF_20211105T221830Z_GMO_00000000326366_01', 'FILE_MAF_20210805T225951Z_GMO_00000000282894_01', 'FILE_MAF_20211105T221624Z_GMO_00000000296344_01', 'FILE_MAF_20220214T004901Z_GMO_00000000144043_01', 'FILE_MAF_20211107T155325Z_GMO_00000000344447_01', 'FILE_MAF_20220215T024650Z_GMO_00000000293219_01', 'FILE_MAF_20211103T110009Z_GMO_00000000338789_01', 'FILE_MAF_20220215T084154Z_GMO_00000000370430_01', 'FILE_MAF_20211222T061241Z_GMO_00000000295406_01', 'FILE_MAF_20211104T120706Z_GMO_00000000378652_01', 'FILE_MAF_20210806T004607Z_GMO_00000000318755_01']

In [19]:
scores=[]
for ep in tqdm(listofepisodes):
    index=materialID.index(ep)
    segmentscore=[]
    for i in tqdm(range(len(newmetalist[index]))):
        res=[]
        for j in range(len(targetlist)):

            if len(newmetalist[index][i])==0:
                continue
            
            doc_ref = nlp(targetlist[0])
            org_doc=nlp(newmetalist[index][i])
            sim_score = org_doc.similarity(doc_ref)
            res.append(sim_score)
        try:
            segmentscore.append(res[17])
        except Exception as e:
            print(e)
            segmentscore.append(0)

    avg=(sum(segmentscore)/len(segmentscore))
    scores.append(avg)
    
print(scores)

  0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

...

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

[0.5351631397028651, 0.5551542428349395, 0.5815448681981471, 0.5366936875308924, 0.5346688378610583, 0.5111839718395771, 0.5752887294781683, 0.5743471031604965, 0.5292423039077483, 0.5985180782675991, 0.5947518597617196, 0.5467691043632321, 0.5750793231740189, 0.5522498803292881, 0.556416860709697, 0.529623820513905, 0.5574795681673892, 0.5171216933860913, 0.4569720709751947, 0.560342926340332, 0.5943713549289446, 0.5990528475136525, 0.5756880744826187, 0.5821196040635582, 0.5728874965635381, 0.5438553597421154, 0.550533752948854, 0.5519850246929272, 0.5717431824457941, 0.5403760342986573, 0.6043149957484512]


In [ ]:
topchef=['FILE_MAF_20220213T082327Z_GMO_00000000030342_01']

In [ ]:
fine_art=['FILE_MAF_20210805T224839Z_GMO_00000000296761_01']

In [ ]:
religion=['FILE_MAF_20220212T185028Z_GMO_00000000007263_01']

In [ ]:
print(listofepisodes[8])

In [ ]:
print(listofepisodes.index('FILE_MAF_20220213T082327Z_GMO_00000000030342_01'))

In [ ]:
#Top Chef aggregation TOP1
topchefscores=[]
for ep in tqdm(religion):
    index=materialID.index(ep)
    segmentscore=[]
    for i in tqdm(range(len(newmetalist[index]))):
        res=[]
        for j in range(len(targetlist)):

            if len(newmetalist[index][i])==0:
                continue
            
            doc_ref = nlp(targetlist[0])
            sim_score = newmetalist[index][i].similarity(doc_ref)
            res.append(sim_score)
        segmentscore.append(res[17])
                
    avg=(sum(segmentscore)/len(segmentscore))
    topchefscores.append(avg)
    
print(topchefscores)

In [ ]:
(sum(scores)-0.5522498803292881)/len(scores)

In [ ]:
0.5522498803292881/0.5391383844387799

In [74]:
en1=model.encode("Wrestling.")
en2=model.encode("Sports")
cos_sim=util.cos_sim(en1,en2)
print(cos_sim)

tensor([[0.6550]])
